任务举例:降水nc数据转tif
坐标系为WGS 84。
工具准备:Python GDAL和netCDF4环境。
操作：替换代码中的属性变量名和路径即可。

In [2]:
import os
import netCDF4 as nc
import numpy as np
from osgeo import gdal, osr, ogr
import glob

In [3]:
def nc2tif(data, Output_folder):
    gpp_data = nc.Dataset(data)  # 利用.Dataset()读取nc数据

    Lat_data = gpp_data.variables['lat'][:]
    # print(Lat_data)
    Lon_data = gpp_data.variables['lon'][:]
    # print(Lon_data)

    #pre_arr= np.asarray(pre_data.variables['gpp'])#属性变量名
    gpp_arr= gpp_data['gpp']#属性变量名
    gpp_mean=np.mean(gpp_arr,axis=0)*31536000000 #时间平均     论文的单位 g/(m^2 year)   数据单位 kg/(m^2 s)
    print(gpp_arr.shape)
    print(gpp_mean.shape)

    # 影像的左上角&右下角坐标
    Lonmin, Latmax, Lonmax, Latmin = [Lon_data.min(), Lat_data.max(), Lon_data.max(), Lat_data.min()]
    # Lonmin, Latmax, Lonmax, Latmin


    # 分辨率计算
    Num_lat = len(Lat_data)  
    Num_lon = len(Lon_data)  
    Lat_res = (Latmax - Latmin) / (float(Num_lat) - 1)
    Lon_res = (Lonmax - Lonmin) / (float(Num_lon) - 1)
    # print(Num_lat, Num_lon)
    # print(Lat_res, Lon_res)


    # for i in range(len(gpp_mean[:])):
        # i=0,1,2,3,4,5,6,7,8,9,...
        # 创建tif文件
    driver = gdal.GetDriverByName('GTiff')
    out_tif_name = Output_folder + '\\' + data.split('\\')[-1][:-3] + '_' + str(1) + '.tif'
    out_tif = driver.Create(out_tif_name, Num_lon, Num_lat, 1, gdal.GDT_Int16)

    # 设置影像的显示范围
    # Lat_re前需要添加负号
    geotransform = (Lonmin, Lon_res, 0.0, Latmax, 0.0, -Lat_res)
    out_tif.SetGeoTransform(geotransform)

    # 定义投影
    prj = osr.SpatialReference()
    prj.ImportFromEPSG(4326)
    out_tif.SetProjection(prj.ExportToWkt())

    # 数据导出
    gpp_mean[gpp_mean < 10**-10] = np.NaN
    gpp_mean = np.flip(gpp_mean, 0)
    out_tif.GetRasterBand(1).WriteArray(gpp_mean)  # 将数据写入内存
    out_tif.FlushCache()  # 将数据写入到硬盘
    out_tif = None  # 关闭tif文件

In [4]:
def main():
    Input_folder = r'G:/lywzh/2023-1-2data/'
    Output_folder = r'G:/lywzh/2023-1-2data/'
    data_list = glob.glob(os.path.join(Input_folder, '*.nc'))
    data_list.sort()
    # 读取所有数据

    print(data_list)
    for i in range(len(data_list)):
        data = data_list[i]
        nc2tif(data, Output_folder)
        print(data + '转tif成功')

main()


['G:/lywzh/2023-1-2data\\gpp_Lmon_GISS-E2-1-G_historical_r1i1p1f1_gn_200101-201412.nc', 'G:/lywzh/2023-1-2data\\gpp_Lmon_GISS-E2-1-H_historical_r1i1p1f1_gn_200101-201412.nc', 'G:/lywzh/2023-1-2data\\gpp_Lmon_INM-CM4-8_historical_r1i1p1f1_gr1_200001-201412.nc']
(168, 90, 144)
(90, 144)
G:/lywzh/2023-1-2data\gpp_Lmon_GISS-E2-1-G_historical_r1i1p1f1_gn_200101-201412.nc转tif成功
(168, 90, 144)
(90, 144)
G:/lywzh/2023-1-2data\gpp_Lmon_GISS-E2-1-H_historical_r1i1p1f1_gn_200101-201412.nc转tif成功
(180, 120, 180)
(120, 180)
G:/lywzh/2023-1-2data\gpp_Lmon_INM-CM4-8_historical_r1i1p1f1_gr1_200001-201412.nc转tif成功
